In [1]:
from wilcoxon import *
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from bs4 import BeautifulSoup

In [34]:
def chineseSentence(word, meanings=''):
    try:
        site = spiderman.website('https://chinesepod.com/dictionary/english-chinese/' + word)
        site.getTables(header=False)
        meanings = re.findall('[A-Za-z]{3,15}',meanings)
        sentenceSets = site.dfs[0][0].tolist()
        sentenceSets = string.findWords(sentenceSets,meanings)
        chosenSentenceSet = sentenceSets[random.randrange(0,len(sentenceSets))]

        sentence = re.split('[\n\t]+',chosenSentenceSet)[1].strip()
        transliteration = re.split('[\n\t]+',chosenSentenceSet)[2].strip()
        translation = re.split('[\n\t]+',chosenSentenceSet)[3].strip()
        
        return [sentence, transliteration, translation]
    except:
        try:
            sentence = re.split('[\n\t]+',chosenSentenceSet)[0].strip()
            transliteration = re.split('[\n\t]+',chosenSentenceSet)[1].strip()
            translation = re.split('[\n\t]+',chosenSentenceSet)[2].strip()

            return [sentence, transliteration, translation]
        except: return ['','','']
        

In [5]:
def tryChineseFullProcess(word, meanings=''):
    L = {'Example':'','Example pinyin':'','Example translation':''}
    L['Word Pinyin'] = string.tryPinyin(word)
    L['Traditional'] = string.tryTraditional(word)
    try: 
        results = chineseSentence(word,meanings)
        try: 
            L['Example'] = results[1]
            L['Example pinyin'] = results[2]
            L['Example translation'] = results[3]
        except:
            L['Example'] = results[0]
            L['Example pinyin'] = results[1]
            L['Example translation'] = results[2]
    except: pass
    return L

In [7]:
vocab.getSheets()
chinese = vocab.getVocab('Chinese')

In [23]:
array = chinese[['Word','Usages','Examples','Example pinyin','Example translation','Word Pinyin','Traditional']].to_numpy()

In [29]:
Ls = []
for row in tqdm(array):
    if pd.isnull(row[3]):
        Ls.append(tryChineseFullProcess(row[0],row[1]))
    else:
        Ls.append({'Example':row[2],'Example pinyin':row[3],'Example translation':row[4],'Word Pinyin':row[5],'Traditional':row[6]})

100%|██████████| 23931/23931 [8:26:35<00:00,  1.27s/it]  


In [11]:
telegram.sendAlarm('Done')

In [28]:
array[112][7]

IndexError: index 7 is out of bounds for axis 0 with size 7

In [30]:
%store Ls

Stored 'Ls' (list)


In [32]:
pd.DataFrame(Ls).to_clipboard()

In [31]:
Ls

[{'Example': '比赛刚刚开始。',
  'Example pinyin': 'bǐsài gānggāng kāishǐ。',
  'Example translation': 'The match just started.',
  'Word Pinyin': 'gāng',
  'Traditional': '剛'},
 {'Example': '大家都喜欢的生肖才不好呢。',
  'Example pinyin': 'dàjiā dōu xǐhuan de shēngxiào cái bù hǎo ne。',
  'Example translation': 'The popular zodiac signs are actually the worst.',
  'Word Pinyin': 'dàjiādū',
  'Traditional': '大傢都'},
 {'Example': '感冒很影响情绪。',
  'Example pinyin': 'gǎnmào hěn yǐngxiǎng qíngxù。',
  'Example translation': "Having the flu really affects one's mood.",
  'Word Pinyin': 'yǐngxiǎng',
  'Traditional': '影響'},
 {'Example': '受台风影响，部分航班延误。',
  'Example pinyin': 'shòu táifēng yǐngxiǎng，bùfen hángbān yánwù。',
  'Example translation': 'Due to the affect cause by typhoon, some flights are delayed.',
  'Word Pinyin': 'yǐngxiǎng',
  'Traditional': '影響'},
 {'Example': '请你把衣服还给我。',
  'Example pinyin': 'qǐng nǐ bǎ yīfu huángěi wǒ。',
  'Example translation': 'Please return the clothing to me.',
  'Word Pinyin': 'huá

In [80]:
df = pd.DataFrame(Ls)[[0,1,2]]
# df[1] = df[0].apply(tryPinyin)

KeyError: "None of [Int64Index([0, 1, 2], dtype='int64')] are in the [columns]"

In [6]:
tryChineseFullProcess('厚','Deep, profound, rich, thick')

{'Example': '还没人来呀！快快，大伙快去找床垫、厚被子什么的，堆在前面，以防万一她掉下来！',
 'Example pinyin': 'hái méi rén lái ya！kuàikuài，dàhuǒ kuài qù zhǎo chuángdiàn、hòu bèizi shénme de，duī zài qiánmian，yǐfáng wànyī tā diàoxiàlái！',
 'Example translation': 'Nobody has come yet! Quickly, everybody quickly go find a mattress, thick quilts and stuff like that and pile it in front there, just in case she falls!',
 'Word Pinyin': 'hòu',
 'Traditional': '厚'}

In [39]:
site = spiderman.website('https://chinesepod.com/dictionary/english-chinese/影响')
site.getTables(header=False)
meanings = re.findall('[A-Za-z]{3,15}','To influence, to affect')
sentenceSets = site.dfs[0][0].tolist()
sentenceSets = string.findWords(sentenceSets,meanings)
chosenSentenceSet = sentenceSets[random.randrange(0,len(sentenceSets))]

sentence = re.split('[\n\t]+',chosenSentenceSet)[1].strip()
transliteration = re.split('[\n\t]+',chosenSentenceSet)[2].strip()
translation = re.split('[\n\t]+',chosenSentenceSet)[3].strip()

[sentence, transliteration, translation]

['前段时间他因为一些私事影响了工作。',
 'qián duàn shíjiān tā yīnwèi yīxiē sīshì yǐngxiǎng le gōngzuò。',
 "In the past while some personal issues he's having have been affecting his work."]

In [99]:
site = BeautifulSoup(open('Documents/Data/Chinese Zero to Hero.txt').read(),'html.parser')

In [118]:
Ls = []
for row in site('tr'):
    link = row.find('a')['href']
    toReturn = [cell.get_text().strip() for cell in row('td')]
    toReturn.append(link)
    Ls.append(toReturn)

In [121]:
pd.DataFrame(Ls).to_clipboard()